# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.experiment import Experiment#



# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-135405
aml-quickstarts-135405
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21


In [16]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'hyper-capstone-1'
project_folder = './capstone-hyper-project'

experiment_hyper = Experiment(ws, experiment_name)
experiment_hyper

Name,Workspace,Report Page,Docs Page
hyper-capstone-1,quick-starts-ws-135405,Link to Azure Machine Learning studio,Link to Documentation


In [18]:

#Create or Attach an AmlCompute cluster

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-hyper"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded...................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [19]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, normal
import os
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core import ScriptRunConfig

#TODO: Create the different params that you will be using during training
### YOUR CODE HERE ###
param_sampling = RandomParameterSampling({
        "--C": choice([0.2,0.4, 0.6, 0.8,1]),
        "--max_iter" : choice ([50,100, 150])
       
    })
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
### YOUR CODE HERE ###
early_termination_policy  = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")
    
if "models" not in os.listdir():
    os.mkdir("./models")

# Create a SKLearn estimator for use with train.py
#TODO: Create your estimator and hyperdrive config
#estimator = SKLearn(source_directory= './', compute_target = 'auto-hyper', entry_script='train.py' )

# Create the environment
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target = compute_target,
                      environment=sklearn_env)
                    



In [20]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=param_sampling ,
                                     run_config=src,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,                 
                                     max_total_runs=4, 
                                     max_concurrent_runs=4, 
                                     policy=early_termination_policy)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [22]:
#TODO: Submit your experiment

hyperdrive_run=experiment_hyper.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_bae3068c-b526-430b-8ec3-927d54d93dd2
Web View: https://ml.azure.com/experiments/hyper-capstone-1/runs/HD_bae3068c-b526-430b-8ec3-927d54d93dd2?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-135405/workspaces/quick-starts-ws-135405

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-19T22:24:35.864286][API][INFO]Experiment created<END>\n""<START>[2021-01-19T22:24:36.613731][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-19T22:24:36.966680][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-19T22:24:37.5490519Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_bae3068c-b526-430b-8ec3-927d54d93dd2
Web View: https://ml.azure.com/experiments/hyper-capstone-1/runs/HD_bae3068c-b526-430b-8ec3-927d54d93dd2?wsid=/subscriptions/cdb

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User errors were found in at least one of the child runs.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User errors were found in at least one of the child runs.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:

import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
#
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameters = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID:   ', best_run.id)
print('\n Accuracy of the best run: ', best_run_metrics['Accuracy'] )

best_run

In [ ]:
best_run.get_file_names()

In [ ]:
#TODO: Save the best model

#save model
model=best_run.register_model('best_model', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service